In [1]:
import pandas as pd
import re

In [2]:
with open('G:\python_github_dump\github_python_dump000000000001', encoding='utf8') as f:
    dump = f.read()

In [3]:
dump = dump.replace('",false,1', '')
splitted = re.split(r'[a-z0-9]{40},\d+,"', dump)
splitted = splitted[1:]

In [115]:
print(splitted[15])

#!/usr/bin/env python
# -*- coding: utf-8 -*-

import json
import os, sys
import itertools
import time
import codecs
from clean_data import Cleaner

# the valid uptodate event_bulletinurl.csv in OUT_DATA folder
LINK_TO_EVENT=""event_bulletinurl.csv""
CLEANING=""cleaning_sections_metadata.json""

#ENB_DATA = ""enb_section_jsons""
ENB_DATA = ""enb_section_jsons-topiked""
OUT_DATA = ""metadata_overview""
if len(sys.argv) > 1:
    ENB_DATA = sys.argv[1]
if len(sys.argv) > 2:
    OUT_DATA = sys.argv[2]
if len(sys.argv) > 3:
    LINK_TO_EVENT = sys.argv[3]
if len(sys.argv) > 4:
    CLEANING = sys.argv[4]

format_txt = lambda x: ('""' + x.replace('""', '""""').replace(""\n"", "" "").replace(""\r"", """") + '""').encode('utf-8')
format_spe = lambda x: format_txt(""|"".join(x))
format_field = lambda x: format_txt(x) if type(x) == unicode else format_spe(x)

data_index={}

# prepare index for LINK SECTIONS TO EVENT task
add_eventid=False
if os.path.exists(os.path.join(OUT_DATA,LINK_TO_EVENT)):
 

In [5]:
def remove_parentheses(input_string):
    parens = []
    opening_p = []
    for index, ch in enumerate(input_string):
        if ch == '(':
            opening_p.append(index)
        elif ch == ')':
            op = opening_p.pop()
            if len(opening_p) == 0:
                parens.append(input_string[op:index+1])
    for p in parens:
        input_string = input_string.replace(p, '')
    return input_string
    

In [6]:
def parse_imports(imports):
    '''Returns list of tuples containing import name, import search string pairs'''
    imports_parsed = set()
    for import_line in imports:
        import_v_from = re.split(r' +', import_line)[0]
        if import_v_from == 'import':
            libs = re.split(r' ?, ?', import_line.replace('import', '').strip())
            for l in libs:
                if ' as ' in l:
                    lib_name = re.split(r' +as +', l)[0].strip()
                    lib_alias = re.split(r' +as +', l)[1].strip()
                    imports_parsed.add((lib_name, lib_alias))
                else:
                    imports_parsed.add((l.strip(), l.strip()))
        elif import_v_from == 'from':
            lib_name = import_line.split('import')[0].replace('from ', '').strip()
            methods_objects = re.split(r' ?, ?', import_line.split('import')[-1].strip())
            for mo in methods_objects:
                if '*' == mo:
                    continue
                if ' as ' in mo:
                    mo_name = re.split(r' +as +', mo)[0].strip()
                    mo_alias = re.split(r' +as +', mo)[1].strip()
                    imports_parsed.add((f'{lib_name}.{mo_name}', mo_alias))
                else:
                    imports_parsed.add((f'{lib_name}.{mo.strip()}', mo.strip()))
    return imports_parsed

In [110]:
def balanced_parentheses(input_string):
    return input_string.count('(') == input_string.count(')')

In [111]:
def remove_comments_docstrings(input_string):
    input_string = re.sub(r'#.*\n', '\n', input_string)
    start = None
    for match in re.finditer(r'["""+|\'\'\'+]+', input_string):
        if not start:
            start = match.span()[0]
        else:
            to_replace = input_string[start: match.span()[1]]
            input_string = input_string.replace(to_replace, len(to_replace)*' ')
            start = None
    return input_string

In [122]:
def find_objects(input_string, imports_list):
    objects = set()
    for imp in imports_list:
        for f in re.findall(r'.+= *{}'.format(imp[1]), input_string):
            objects.add((f'{imp[0]}_derivative', f.split('=')[0].strip()))
    return objects

In [ ]:
def count_usage(input_string, names):
    for line in input_string.replace('\\\n', ' ').splitlines():
        

In [123]:
imports = []
for imp in re.findall(r'\n *import .+|\n *from .+ import .+', splitted[15]):
    import_line = imp.split('#')[0].strip()
    snippet = snippet.replace(import_line, '')
    imports.append(import_line)
imports = parse_imports(imports)
print(find_objects(splitted[15], imports))

{('clean_data_Cleaner_object', 'cleaner'), ('json_object', 'data'), ('sys_object', 'LINK_TO_EVENT'), ('time_object', 'date'), ('sys_object', 'ENB_DATA'), ('time_object', 'year'), ('sys_object', 'CLEANING'), ('sys_object', 'OUT_DATA'), ('time_object', 'epoch_millisecond')}
